In [2]:
#! /usr/bin/python3

import os

import dash
import dash_auth
import dash_core_components as dcc
import dash_html_components as html
import dateutil
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime
import sys
import os


ModuleNotFoundError: No module named 'dash.dcc'

In [1]:
VALID_USERNAME_PASSWORD_PAIRS = [
    ['Angra', 'Angra@Now!']
]

#scale = 1000/250
scale = 1

In [2]:

def tail(file,lines):
    iter = 0
    bufsize = 8192
    fsize = os.stat(file).st_size
    with open(file) as f:
        if bufsize > fsize:
            bufsize = fsize-1
        data = []
        while True:
            iter +=1
            f.seek(fsize-bufsize*iter)
            data.extend(f.readlines())
            if len(data) >= lines or f.tell() == 0:
                return (data[-lines:])



In [3]:
app = dash.Dash()

auth = dash_auth.BasicAuth(
    app,
    VALID_USERNAME_PASSWORD_PAIRS
)

app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})

pmts = []
num = []
for i in range(1,33):
    pmts.append('PMT'+str(i))
    num.append(0)

num2 = []
for i in range(0,8):
    num2.append(0)

#HTML Layout
app.layout = html.Div(children=[
    html.H1(children='Neutrinos Angra Dashboard', style={'textAlign': 'center'}),
    html.Div([
        dcc.Graph(id='my-graph')], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='my-graph2')], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='my_bars')], style={'width': '74%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='my_bars2')], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='my-graph-temp1')], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id='my-graph-temp2')], style={'width': '49%', 'display': 'inline-block'}),
    dcc.Interval(id='my-graph-update', interval=2500, n_intervals=0),

])


NameError: name 'dash' is not defined

In [22]:
#Interactions
@app.callback(dash.dependencies.Output('my-graph', 'figure'), [dash.dependencies.Input('my-graph-update', 'n_intervals')])
def update_scatterplot(interval):
    data = tail('/home/lsd/trigger/DATA_LOG.LOG',21)
    freq = []
    time = []
    for i in range(0,20):
        f = float(int(data[i+1].split(' ')[-5])-int(data[i].split(' ')[-5]))/(float(data[i+1].split(' ')[-1])-float(data[i].split(' ')[-1]))
        if f < 0:
            f = 0
        freq.append(f)
        time.append(datetime.datetime.fromtimestamp(float(data[i].split(' ')[-1][:-1])))
    return {
        'data': [
            go.Scatter(
                x=time,
                y=freq,
                text="",
                mode='lines',
                opacity=0.7,
                marker={
                    'size': 15,
                    'color': 'blue',
                    'line': {'width': 0.5, 'color': 'blue'}
                }
            )
        ],
        'layout': go.Layout(
            title = 'Trigger Rate',
            titlefont=dict(family='Arial, sans-serif',size=24,color='blue'),
            xaxis={'title': 'Time'},
            yaxis={'title': 'Frequency [Hz]'},
            margin={'l': 45, 'b': 40, 't': 50, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }


In [23]:
@app.callback(dash.dependencies.Output('my-graph2', 'figure'), [dash.dependencies.Input('my-graph-update', 'n_intervals')])
def update_scatterplot(interval):
    data = tail('/home/lsd/trigger/DATA_LOG.LOG',21)
    freq = []
    time = []
    for i in range(0,20):
        f = float(int(data[i+1].split(' ')[-4])-int(data[i].split(' ')[-4]))/(float(data[i+1].split(' ')[-1])-float(data[i].split(' ')[-1]))
        if f < 0:
            f = 0
        freq.append(f)
        time.append(datetime.datetime.fromtimestamp(float(data[i].split(' ')[-1][:-1])))
    return {
        'data': [
            go.Scatter(
                x=time,
                y=freq,
                text="",
                mode='lines',
                opacity=0.7,
                marker={
                    'size': 15,
                    'color': 'red',
                    'line': {'width': 0.5, 'color': 'blue'}
                }
            )
        ],
        'layout': go.Layout(
            title = 'Veto Rate',
            titlefont=dict(family='Arial, sans-serif',size=24,color='red'),
            xaxis={'title': 'Time'},
            yaxis={'title': 'Frequency [Hz]'},
            margin={'l': 50, 'b': 40, 't': 50, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }


In [24]:
@app.callback(dash.dependencies.Output('my_bars', 'figure'), [dash.dependencies.Input('my-graph-update', 'n_intervals')])
def update_bar_chart(interval):
    global scale
    data = tail('/home/lsd/trigger/DATA_LOG.LOG',1)
    x = int(data[0].split(' ')[13])
    #num = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    group = x>>68
    color = []
    #print('Grupo: ' + str(group))
    if group == 20:
        for i in range (0,2):
            num[group*4+i]=((x>>16*i)&0xFFFF)*scale
    elif group < 8:
        for i in range (0,4):
            num[group*4+i]=((x>>16*i)&0xFFFF)*scale
            #print('PMTs: ' + str(group*4+i))
            #print('Grupo: ' + str(group))
    np_num = np.array(num)
    color=np.array(['blue']*np_num.shape[0])
    color[np_num>6000]='gray'
    return {
        'data': [
            go.Bar(
                #x=pmts,
                x=['PMT1 (SD2805)','PMT2 (SD2758)','PMT3 (SD2804)','PMT4 (SD2772)',\
                'PMT5 (SD2774)','PMT6 (SD2762)','PMT7 (SD2843)','PMT8 (SD2779)',\
                'PMT9 (SD2856)','PMT10 (SD2854)','PMT11 (SD2792)','PMT12 (SD2851)',\
                'PMT13 (SD2837)','PMT14 (SD2777)','PMT15 (SD2842)','PMT16 (SD2763)',\
                'PMT17 (SD2806)','PMT18 (SD2855)','PMT19 (SD2836)','PMT20 (SD2857)',\
                'PMT21 (SD2850)','PMT22 (SD2801)','PMT23 (SD2794)','PMT24 (SD2786)',\
                'PMT25 (SD2759)','PMT26 (SD2784)','PMT27 (SD2847)','PMT28 (SD2789)',\
                'PMT29 (SD2778)','PMT30 (SD2790)','PMT31 (SD2770)','PMT32 (SD2787)'],
                y=num,
                marker=dict(color=color.tolist())
            )
        ],
        'layout': go.Layout(
            title = 'Trigger Rate of Target PMTs',
            titlefont=dict(family='Arial, sans-serif',size=24,color='blue'),
            yaxis=dict(title = 'Trigger Rate (Hz)'),#,range=[0, 6000]),
            barmode='stack',
            hovermode='closest',
        )
    }


In [25]:
@app.callback(dash.dependencies.Output('my_bars2', 'figure'), [dash.dependencies.Input('my-graph-update', 'n_intervals')])
def update_bar_chart(interval):
    global scale
    data = tail('/home/lsd/trigger/DATA_LOG.LOG',1)
    x = int(data[0].split(' ')[13])
    group = x>>68
    if group == 8:
        for i in range (0,4):
            num2[i]=((x>>16*i)&0xFFFF)*scale
    elif group == 9:
        for i in range (0,4):
            num2[i+4]=((x>>16*i)&0xFFFF)*scale

    return {
        'data': [
            go.Bar(
                x=['SD2791','SD2785','SD2782','SD2849','ZS0057','SD2975','SD2968','SD2771'],
                y=num2,
                marker={
                    'color': 'red'
                }
            )
        ],
        'layout': go.Layout(
            title = 'Trigger Rate of Veto PMTs',
            titlefont=dict(family='Arial, sans-serif',size=24,color='red'),
            yaxis={'title': 'Trigger Rate (Hz)'},
            barmode='stack',
            hovermode='closest'
        )
    }


In [26]:
@app.callback(dash.dependencies.Output('my-graph-temp1', 'figure'), [dash.dependencies.Input('my-graph-update', 'n_intervals')])
def update_scatterplot(interval):
    data = tail('/home/lsd/trigger/DATA_LOG.LOG',21)
    freq = []
    time = []
    for i in range(0,20):
        f = (int(data[i+1].split(' ')[-3]) - int(data[i].split(' ')[-3]))
        f = int(f) & 0xFFFFFFF
        f = f / (float(data[i+1].split(' ')[-1])-float(data[i].split(' ')[-1]))
        if f < 0:
            f = 0
        freq.append(f)
        time.append(datetime.datetime.fromtimestamp(float(data[i].split(' ')[-1][:-1])))
    return {
        'data': [
            go.Scatter(
                x=time,
                y=freq,
                text="",
                mode='lines',
                opacity=0.7,
                marker={
                    'size': 15,
                    'color': 'black',
                    'line': {'width': 0.5, 'color': 'black'}
                }
            )
        ],
        'layout': go.Layout(
            title = 'AND of 8 Target channels',
            titlefont=dict(family='Arial, sans-serif',size=24,color='black'),
            xaxis={'title': 'Time'},
            yaxis={'title': 'Count Rate [Hz]'},
            margin={'l': 45, 'b': 40, 't': 50, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }


In [27]:
@app.callback(dash.dependencies.Output('my-graph-temp2', 'figure'), [dash.dependencies.Input('my-graph-update', 'n_intervals')])
def update_scatterplot(interval):
    data = tail('/home/lsd/trigger/DATA_LOG.LOG',21)
    freq = []
    time = []
    for i in range(0,20):
        f = (int(data[i+1].split(' ')[-3]) - int(data[i].split(' ')[-3]))
        f = int(f) >> 28
        f = f / (float(data[i+1].split(' ')[-1])-float(data[i].split(' ')[-1]))
        if f < 0:
            f = 0
        freq.append(f)
        time.append(datetime.datetime.fromtimestamp(float(data[i].split(' ')[-1][:-1])))
    return {
        'data': [
            go.Scatter(
                x=time,
                y=freq,
                text="",
                mode='lines',
                opacity=0.7,
                marker={
                    'size': 15,
                    'color': 'black',
                    'line': {'width': 0.5, 'color': 'black'}
                }
            )
        ],
        'layout': go.Layout(
            title = 'AND of the 4 TopVeto channels',
            titlefont=dict(family='Arial, sans-serif',size=24,color='black'),
            xaxis={'title': 'Time'},
            yaxis={'title': 'Count Rate [Hz]'},
            margin={'l': 45, 'b': 40, 't': 50, 'r': 10},
            legend={'x': 0, 'y': 1},
            hovermode='closest'
        )
    }


In [28]:
if __name__ == '__main__':
    port = int(os.environ.get('PORT', 1000))
    debug = os.environ.get('PRODUCTION') is None
    app.run_server(debug=debug, host='0.0.0.0', port=port, ssl_context='adhoc')


UnsupportedOperation: not writable